In [2]:
import numpy as np

class Variable():
    def __init__(self, data):
        self.data = data

data = np.array(1.0)
x = Variable(data)
print(x.data)

1.0


In [3]:
x.data = np.array(2.0)
print(x.data)

2.0


In [4]:
class Function():
    def __call__(self, input):
        x = input.data
        y = x**2
        output = Variable(y)
        return output

In [5]:
x = Variable(np.array(10))
f = Function()
y = f(x)

print(type(y))
print(y.data)

<class '__main__.Variable'>
100


In [13]:
# 様々な関数を使えるようにfunctionを作り替える
class Function():
    def __call__(self, input):
        x = input.data
        y = self.forward(x)
        output = Variable(y)
        return output

    def forward(self, x):
        raise NotImplementedError()

In [21]:
class Square(Function):
    def forward(self, x):
        return x**2

In [22]:
x = Variable(np.array(10))
f = Square()
y = f(x)
print(type(y))
print(y.data)

<class '__main__.Variable'>
100


In [23]:
# y = e^xを実装
class Exp(Function):
    def forward(self, x):
        return np.exp(x)

In [24]:
A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)
print(y.data)

1.648721270700128


In [25]:
# 数値微分を実装
def numerical_diff(f, x, eps = 1e-4):
    x0 = Variable(x.data - eps)
    x1 = Variable(x.data + eps)
    y0 = f(x0)
    y1 = f(x1)
    return (y1.data - y0.data)/(2*eps)

In [26]:
f = Square()
x = Variable(np.array(2.0))
dy = numerical_diff(f, x)
print(dy)

4.000000000004


In [29]:
# 合成関数の微分
def f(x):
    A = Square()
    B = Exp()
    C = Square()
    return C(B(A(x)))

x = Variable(np.array(0.5))
dy = numerical_diff(f, x)
print(dy)

3.2974426293330694


# step6 手作業によるバックプロぱゲーション 

In [31]:
# 逆誤差伝播に対応したvariable
class Variable():
    def __init__(self, data):
        self.data = data
        self.grad = None

In [32]:
class Function():
    def __call__(self, input):
        x = input.data
        y = self.forward(x)
        output = Variable(y)
        self.input = input # 入力された変数を覚える
        return output
    
    def forward(self, x):
        raise NotImplementedError()

    def backward(self, gy):
        raise NotImplementedError()

In [33]:
class Square(Function):
    def forward(self, x):
        y = x ** 2
        return y
    
    def backward(self, gy):
        x = self.input.data # functionで保存したやつ
        gx = 2 * x * gy
        return gx

In [35]:
class Exp(Function):
    def forward(self, x):
        y = np.exp(x)
        return y

    def backward(self, gy):
        x = self.input.data
        gx = np.exp(x) * gy
        return gx

In [37]:
# 順伝播のテスト
A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)
print(y.data)

1.648721270700128


In [38]:
# 逆伝播
y.grad = np.array(1.0)
b.grad = C.backward(y.grad)
a.grad = B.backward(b.grad)
x.grad = A.backward(a.grad)
print(x.grad)

3.297442541400256


# ステップ7 バックプロパゲーションの自動化

In [39]:
# 変数と関数のつながりを明示的に入れ込む
class Variable():
    def __init__(self, data):
        self.data = data
        self.grad = None
        self.creator = None
    
    def set_creator(self, func):
        self.creator = func # この変数の生みの親はfuncですという意味

In [41]:
class Function():
    def __call__(self, input):
        x = input.data
        y = self.forward(x)
        output = Variable(y)
        output.set_creator(self) # 生みの親を覚えさせる
        self.input = input
        self.output = output # outputも覚えさせる
        return output

In [43]:
class Square(Function):
    def forward(self, x):
        y = x ** 2
        return y
    
    def backward(self, gy):
        x = self.input.data # functionで保存したやつ
        gx = 2 * x * gy
        return gx

In [44]:
class Exp(Function):
    def forward(self, x):
        y = np.exp(x)
        return y

    def backward(self, gy):
        x = self.input.data
        gx = np.exp(x) * gy
        return gx

In [45]:
# 順伝播のテスト
A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)

In [49]:
y.grad = np.array(0.5)

C = y.creator # 関数を取得
b = C.input # 関数の入力を取得
b.grad = C.backward(y.grad)

In [50]:
# 逆伝播を自動化
class Variable():
    def __init__(self, data):
        self.data = data
        self.grad = None
        self.creator = None
    
    def set_creator(self, func):
        self.creator = func # この変数の生みの親はfuncですという意味

    def backward(self):
        f = self.creator # 関数を取得
        if f is not None:
            x = f.input
            x.grad = f.backward(self.grad)
            x.backward()
    

In [51]:
A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)

# backward
y.grad = np.array(1.0)
y.backward()
print(x.grad)

3.297442541400256


y.backwardで実行してみると…  

- f = y.creator → C  
- x = C.input → b  
- b.grad = C.backward(y.grad)  
- b.backward  

# ステップ8:再帰からループへ

In [65]:
# variableを再帰的な処理に置き換える
class Variable():
    def __init__(self, data):
        self.data = data
        self.grad = None
        self.creator = None
    
    def set_creator(self, func):
        self.creator = func # この変数の生みの親はfuncですという意味

    def backward(self):
        funcs = [self.creator] # リスト形式に変更
        while funcs:
            f = funcs.pop() # リストの最後をとって関数を取得
            x, y = f.input, f.output
            x.grad = f.backward(y.grad)

            if x.creator is not None: # xのcreatorがいるなら
                funcs.append(x.creator) # リストに関数を追加。また取り出して…が続く

# ステップ9:関数をより便利に

In [66]:
x = Variable(np.array(0.5))
f = Square()
y = f(x)
y.data

0.25

二乗の計算を行うのにわざわざインスタンス作って呼び出すのめんどくさい…

In [67]:
# 関数にしてしまおう
def square(x):
    f = Square()
    return f(x)

def exp(x):
    f = Exp()
    return f(x)

In [68]:
x = Variable(np.array(0.5))
square(x).data

0.25

In [69]:
# y.grad = np.array(1.0)を書くのが面倒なのでbackwardメソッドに追加しておく
class Variable():
    def __init__(self, data):
        self.data = data
        self.grad = None
        self.creator = None
    
    def set_creator(self, func):
        self.creator = func # この変数の生みの親はfuncですという意味

    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data) # gradがなかったら同じ形状の1インスタンスを作成

        funcs = [self.creator] # リスト形式に変更
        while funcs:
            f = funcs.pop() # リストの最後をとって関数を取得
            x, y = f.input, f.output
            x.grad = f.backward(y.grad)

            if x.creator is not None: # xのcreatorがいるなら
                funcs.append(x.creator) # リストに関数を追加。また取り出して…が続く
    

In [70]:
# かなり書くのが楽になったことを実感
x = Variable(np.array(0.5))
y =square(exp(square(x)))
y.backward()
print(x.grad)


3.297442541400256


In [72]:
# variableにndarray以外を与えられた時即座にエラーを吐くようにする
class Variable():
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError('{} is not supported'.format(type(data)))
        self.data = data
        self.grad = None
        self.creator = None
    
    def set_creator(self, func):
        self.creator = func # この変数の生みの親はfuncですという意味

    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data) # gradがなかったら同じ形状の1インスタンスを作成

        funcs = [self.creator] # リスト形式に変更
        while funcs:
            f = funcs.pop() # リストの最後をとって関数を取得
            x, y = f.input, f.output
            x.grad = f.backward(y.grad)

            if x.creator is not None: # xのcreatorがいるなら
                funcs.append(x.creator) # リストに関数を追加。また取り出して…が続く

In [74]:
# ちゃんとエラーを吐く
x = Variable(0.5)

TypeError: <class 'float'> is not supported

In [75]:
# 0次元だと計算後にfloatになってしまう仕様があるので、回避する
def as_array(x):
    if np.isscaler(x):
        return np.array(x)
    return x

In [76]:
class Function():
    def __call__(self, input):
        x = input.data
        y = self.forward(x)
        output = Variable(as_array(y))
        output.set_creator(self) # 生みの親を覚えさせる
        self.input = input
        self.output = output # outputも覚えさせる
        return output

# ステップ10:テストを行う

In [78]:
# pythonのユニットテスト
import unittest

class SquareTest(unittest.TestCase):
    def test_forward(self):
        x = Variable(np.array(2.0))
        y = square(x)
        expected = np.array(4.0)
        self.assertEqual(y.data, expected)
